# Performance Evaluation - Portfolio Managers Analysis

**Problem 3:** You run a large university endowment that invests with outside portfolio managers. Two final candidates have been presented to you. The PMs' track records from 2005 to 2024 are included in the spreadsheet "PM.xlsx." You might also need the data for monthly market excess returns, returns for the SMB and HML factors, and one-month Treasury bill rates from Ken French's website.


## Data Loading and Setup


In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 6)

# Load data
manager_df = pd.read_excel('PM.xlsx')
ff3_df = pd.read_csv('F-F_Research_Data_Factors.csv')

# Process dates and merge data
ff3_df['Date'] = pd.to_datetime(ff3_df['Date'], format='%Y%m')
manager_df['Date'] = pd.to_datetime(manager_df['Date'], format='%Y%m')
ff3_df = ff3_df.set_index('Date')
manager_df = manager_df.set_index('Date')

# Convert to decimal format
manager_df /= 100
ff3_df /= 100

# Merge datasets
df = pd.merge(left=manager_df, right=ff3_df, left_index=True, right_index=True)

print(f"Data loaded: {df.shape[0]} observations from {df.index.min().strftime('%Y-%m')} to {df.index.max().strftime('%Y-%m')}")


## (a) Summarize the performances of the two managers

### Scatter plots of managers' monthly excess returns against market excess returns


In [ ]:
# Calculate excess returns
df['PM1_excess'] = df['PM1'] - df['RF']
df['PM2_excess'] = df['PM2'] - df['RF']

# Create scatter plot
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.scatter(df['Mkt-RF'], df['PM1_excess'], alpha=0.6, color='red')
plt.xlabel('Market Excess Return')
plt.ylabel('PM1 Excess Return')
plt.title('PM1 vs Market Excess Returns')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.scatter(df['Mkt-RF'], df['PM2_excess'], alpha=0.6, color='blue')
plt.xlabel('Market Excess Return')
plt.ylabel('PM2 Excess Return')
plt.title('PM2 vs Market Excess Returns')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### Boxplot of the two managers' monthly returns


In [ ]:
plt.figure(figsize=(8, 6))
plt.boxplot([df['PM1'], df['PM2']], tick_labels=['PM1', 'PM2'])
plt.title('Side-by-Side Boxplots of PM1 and PM2 Monthly Returns')
plt.ylabel('Monthly Returns')
plt.grid(True, alpha=0.3)
plt.show()


### Performance Metrics: Mean, Standard Deviation, Sharpe Ratio, Information Ratio, and Maximum Drawdown


In [ ]:
def calculate_performance_metrics(returns, rf_rate, market_excess):
    """Calculate performance metrics for a portfolio manager"""
    excess_returns = returns - rf_rate
    excess_vs_market = returns - market_excess - rf_rate
    
    # Basic statistics
    mean_return = returns.mean()
    std_return = returns.std()
    
    # Sharpe ratio
    sharpe_ratio = excess_returns.mean() / excess_returns.std()
    
    # Information ratio (excess return over market / tracking error)
    info_ratio = excess_vs_market.mean() / excess_vs_market.std()
    
    # Maximum drawdown calculation
    cumulative_returns = (1 + returns).cumprod()
    running_max = cumulative_returns.expanding().max()
    drawdown = (cumulative_returns - running_max) / running_max
    max_drawdown = drawdown.min()
    
    return {
        'Mean': mean_return,
        'Std Dev': std_return,
        'Sharpe Ratio': sharpe_ratio,
        'Information Ratio': info_ratio,
        'Max Drawdown': max_drawdown
    }

# Calculate metrics for both managers
pm1_metrics = calculate_performance_metrics(df['PM1'], df['RF'], df['Mkt-RF'])
pm2_metrics = calculate_performance_metrics(df['PM2'], df['RF'], df['Mkt-RF'])

# Display results
results_df = pd.DataFrame({'PM1': pm1_metrics, 'PM2': pm2_metrics})
print("Performance Metrics Summary:")
print("=" * 40)
print(results_df.round(4))


### Observations on Differences and Outliers

**Key Differences between PM1 and PM2:**

1. **Volatility**: PM1 exhibits much higher variance in returns compared to PM2, indicating higher risk.

2. **Return Distribution**: Both managers show similar mean returns, but PM2 has significantly more outliers, particularly in the negative direction.

3. **Risk-Return Profile**: PM1 shows higher volatility but also potentially higher returns, while PM2 appears more conservative with tighter return distribution.

4. **Outlier Treatment**: The presence of outliers, especially for PM2, suggests that winsorization might be considered to remove extreme high and low returns for more robust analysis.


## (b) Evidence for ability to outperform the market using Fama-French 3-factor model

### Fama-French 3-Factor Model: R*ᵢ,ₜ = αᵢ + βᵢRₘ,ₜ + γᵢRₕₘₗ,ₜ + δᵢRₛₘᵦ,ₜ + εᵢ,ₜ

### PM1 Regression Results


In [ ]:
# PM1 Fama-French 3-factor regression
Y_pm1 = df['PM1'] - df['RF']  # PM1 excess returns
X_pm1 = sm.add_constant(df[['Mkt-RF', 'HML', 'SMB']])

model_pm1 = sm.OLS(endog=Y_pm1, exog=X_pm1).fit()
print("PM1 Fama-French 3-Factor Model Results:")
print("=" * 50)
model_pm1.summary()


### PM2 Regression Results


In [ ]:
# PM2 Fama-French 3-factor regression
Y_pm2 = df['PM2'] - df['RF']  # PM2 excess returns
X_pm2 = sm.add_constant(df[['Mkt-RF', 'HML', 'SMB']])

model_pm2 = sm.OLS(endog=Y_pm2, exog=X_pm2).fit()
print("PM2 Fama-French 3-Factor Model Results:")
print("=" * 50)
model_pm2.summary()


## (c) Which PM is better at beating the market? Sharpe ratio and Information ratio analysis

### Summary and Conclusions


In [ ]:
# Extract key statistics for comparison
print("PERFORMANCE COMPARISON SUMMARY")
print("=" * 50)

print(f"\n1. MARKET-BEATING ABILITY (Information Ratio):")
print(f"   PM1 Information Ratio: {pm1_metrics['Information Ratio']:.4f}")
print(f"   PM2 Information Ratio: {pm2_metrics['Information Ratio']:.4f}")
print(f"   Winner: {'PM1' if pm1_metrics['Information Ratio'] > pm2_metrics['Information Ratio'] else 'PM2'}")

print(f"\n2. RISK-ADJUSTED RETURNS (Sharpe Ratio):")
print(f"   PM1 Sharpe Ratio: {pm1_metrics['Sharpe Ratio']:.4f}")
print(f"   PM2 Sharpe Ratio: {pm2_metrics['Sharpe Ratio']:.4f}")
print(f"   Winner: {'PM1' if pm1_metrics['Sharpe Ratio'] > pm2_metrics['Sharpe Ratio'] else 'PM2'}")

print(f"\n3. FAMA-FRENCH 3-FACTOR ALPHA (Ability to generate alpha):")
print(f"   PM1 Alpha: {model_pm1.params['const']:.4f} (t-stat: {model_pm1.tvalues['const']:.3f})")
print(f"   PM2 Alpha: {model_pm2.params['const']:.4f} (t-stat: {model_pm2.tvalues['const']:.3f})")
alpha_winner = 'PM1' if model_pm1.tvalues['const'] > model_pm2.tvalues['const'] else 'PM2'
print(f"   Winner (higher t-statistic): {alpha_winner}")

print(f"\n4. FACTOR EXPOSURES:")
print(f"   PM1 - Market Beta: {model_pm1.params['Mkt-RF']:.3f}, HML: {model_pm1.params['HML']:.3f}, SMB: {model_pm1.params['SMB']:.3f}")
print(f"   PM2 - Market Beta: {model_pm2.params['Mkt-RF']:.3f}, HML: {model_pm2.params['HML']:.3f}, SMB: {model_pm2.params['SMB']:.3f}")


### Final Recommendation

Based on the comprehensive analysis:

**1. Market-Beating Ability**: PM1 demonstrates superior ability to beat the market with a higher Information Ratio.

**2. Risk-Adjusted Performance**: PM1 generates a slightly better Sharpe ratio, indicating better risk-adjusted returns.

**3. Alpha Generation**: PM1 shows stronger evidence of alpha generation with a higher t-statistic in the Fama-French 3-factor model, suggesting more statistically significant outperformance.

**Conclusion**: **PM1 is the better choice** for the university endowment based on superior market-beating ability, better risk-adjusted returns, and stronger statistical evidence of alpha generation through the Fama-French 3-factor analysis.
